<a href="https://colab.research.google.com/github/diegozamorasanchez/03MIAR-Algoritmos-de-Optimizacion/blob/main/Algoritmos_AG3_Rev.ColoniaHormigas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nombre: Diego Zamora Sanchez <br>
Link: https://colab.research.google.com/drive/1zL5nXwOblhWQo8xSTQK-lE5VMZ-0yo3K?usp=sharing <br>
GitHub: https://github.com/diegozamorasanchez/03MIAR-Algoritmos-de-Optimizacion <br>

1. EL PROBLEMA DEL AGENTE VIAJERO

In [89]:
#CARGA DE LIBRERIAS
!pip install requests    #Hacer llamadas http a paginas de la red
!pip install tsplib95    #Modulo para las instancias del problema del TSP

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [90]:
#CARGAMOS LOS DATOS DEL PROBLEMA
import urllib.request #Para llamadas http
import tsplib95       #Para instancias del problema del TSP
import math           #Funciones matematicas
import random         #Generadores de valores aleatorios

#Descargamos el fichero con la Matriz de Distancias
file = "swiss42.tsp" ; 
urllib.request.urlretrieve("http://comopt.ifi.uni-heidelberg.de/software/TSPLIB95/tsp/swiss42.tsp.gz", file + '.gz')
!gzip -d swiss42.tsp.gz     #Descomprime

gzip: swiss42.tsp already exists; do you wish to overwrite (y or n)? y


In [91]:
#Carga de datos descomprimidas y generación del objeto (problema del viajante)
problem = tsplib95.load(file)
#Nodos
Nodos = list(problem.get_nodes())
#Aristas
Aristas = list(problem.get_edges())

In [92]:
#Probamos algunas funciones del objeto problema

#Distancia entre nodos
problem.get_weight(0, 1)

#Todas las funciones en la documentación: https://tsplib95.readthedocs.io/en/v0.6.1/modules.html               
#dir(problem)

15

In [93]:
#FUNCION PARA GENERAR SOLUCION ALEATORIA
def crear_solucion(Nodos): 
  solucion = [Nodos[0]]
  for n in Nodos[1:]:
    solucion = solucion + [random.choice(list(set(Nodos) - set({Nodos[0]}) - set(solucion)))]
  return solucion 

#FUNCION DISTANCIA ENTRE 2 NODOS
def distancia(a,b, problem):
  return problem.get_weight(a,b)

#FUNCION DISTANCIA TOTAL DE UNA SOLUCION O TRAYECTORIA
def distancia_total(solucion, problem):
  distancia_total = 0
  for i in range(len(solucion)-1):
    distancia_total += distancia(solucion[i] ,solucion[i+1] ,  problem)
  return distancia_total + distancia(solucion[len(solucion)-1] ,solucion[0], problem)

In [94]:
solucion = crear_solucion(Nodos)
solucion

[0,
 33,
 13,
 10,
 20,
 11,
 7,
 17,
 25,
 36,
 28,
 26,
 38,
 29,
 14,
 4,
 34,
 8,
 41,
 35,
 32,
 37,
 5,
 27,
 9,
 40,
 23,
 39,
 2,
 21,
 16,
 18,
 24,
 22,
 12,
 6,
 19,
 3,
 1,
 30,
 31,
 15]

In [95]:
distancia_total(solucion, problem)

4748

2. BÚSQUEDA ALEATORIA

In [96]:
#FUNCION PRINCIPAL DE BUSQUEDA ALEATORIA
def busqueda_aleatoria(problem, N):
  Nodos = list(problem.get_nodes())

  mejor_solucion=[]
  mejor_distancia=float('inf')

  for i in range(N):
    solucion=crear_solucion(Nodos)
    distancia=distancia_total(solucion, problem)

    if distancia < mejor_distancia:
      mejor_solucion=solucion
      mejor_distancia=distancia

  print("Mejor solución:" , mejor_solucion)
  print("Distancia:" , mejor_distancia)
  return mejor_solucion

In [97]:
#Busqueda aleatoria con N=10000 iteraciones
solucion = busqueda_aleatoria(problem,10000)

Mejor solución: [0, 5, 1, 2, 29, 41, 26, 32, 31, 12, 23, 15, 33, 34, 9, 13, 18, 6, 4, 27, 20, 30, 28, 3, 11, 35, 37, 17, 16, 14, 40, 24, 36, 7, 19, 38, 22, 10, 25, 21, 39, 8]
Distancia: 3762


3. BÚSQUEDA LOCAL

In [98]:
#FUNCION DE BUSQUEDA LOCAL CON GENERADOR DE SOLUCIONES VECINAS
def genera_vecina(solucion):
  mejor_solucion=[]
  mejor_distancia=10e100
  #Recorremos todos los nodos
  for i in range(1,len(solucion)-1):
    for j in range(i+1, len(solucion)):
      
      #a: se genera una nueva solución intercambiando los dos nodos i,j
      vecina = solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

      #b: se evalua la solución
      distancia_vecina=distancia_total(vecina, problem)    

      #c: Se guarda si es mejor
      if distancia_vecina <= mejor_distancia:
        mejor_distancia=distancia_vecina
        mejor_solucion=vecina

  return mejor_solucion

In [99]:
#solucion=[1,47,13,41,40,19,42,44,37,5,22,28,3,2,29,21,50,34,30,9,16,11,38,49,10,39,33,45,15,24,43,26,31,36,35,20,8,7,23,48,27,12,17,4,18,25,14,6,51,46,32]
print("Distancia Solucion Incial:" , distancia_total(solucion, problem))

nueva_solucion=genera_vecina(solucion)
print("Distancia Mejor Solucion Local:", distancia_total(nueva_solucion, problem))

Distancia Solucion Incial: 3762
Distancia Mejor Solucion Local: 3510


In [100]:
#FUNCION DE BUSQUEDA LOCAL SOBRE LA ANTERIOR SIN CRITERIO DE PARADA
def busqueda_local(problem):
  mejor_solucion=[]

  #Solucion inicial aleatoria
  solucion_referencia=crear_solucion(Nodos)
  mejor_distancia=distancia_total(solucion_referencia, problem)

  iteracion=0
  while(1):
    iteracion +=1

    #a: mejor vecina
    vecina=genera_vecina(solucion_referencia)

    #b:se evalúa
    distancia_vecina=distancia_total(vecina, problem)

    #c: se para si no mejoramos (mínimo local)
    if distancia_vecina < mejor_distancia:
      mejor_solucion=vecina
      mejor_distancia=distancia_vecina

    else:
      print("En la iteracion ", iteracion, ", la mejor solución encontrada es:" , mejor_solucion)
      print("Distancia:" , mejor_distancia)
      return mejor_solucion

In [101]:
#Solución búsqueda local:
sol=busqueda_local(problem)

En la iteracion  2 , la mejor solución encontrada es: [0, 3, 33, 24, 5, 11, 4, 7, 21, 26, 20, 15, 37, 13, 28, 40, 10, 18, 27, 34, 25, 29, 31, 38, 23, 36, 14, 16, 35, 30, 41, 32, 9, 22, 39, 2, 1, 8, 6, 17, 19, 12]
Distancia: 4564


4. SIMULATED ANNEALING

In [102]:
#FUNCION GENERADORA DE SOLUCIONES VECINAS 100% ALEATORIA
def genera_vecina_aleatorio(solucion):

  #2 nodos aleatorios
  i,j = sorted(random.sample( range(1,len(solucion)) , 2))
  
  #Devuelve una nueva solución intercambiando los 2 nodos al azar
  return solucion[:i] + [solucion[j]] + solucion[i+1:j] + [solucion[i]] + solucion[j+1:]

#FUNCION PROBABILIDAD ACEPTACION
def probabilidad(T,d):
  if random.random() <  math.exp( -1*d / T):
    return True
  else:
    return False

#FUNCION DESCENSO TEMPERATURA
def bajar_temperatura(T):
  return T*0.99

In [103]:
def recocido_simulado(problem, TEMPERATURA):

  solucion_referencia=crear_solucion(Nodos)
  distancia_referencia=distancia_total(solucion_referencia, problem)
  
  mejor_solucion = []             #x* del seudocodigo
  mejor_distancia = 10e100        #F* del seudocodigo
  
  N=0
  while TEMPERATURA > .0001:
    N+=1
    #Genera solución vecina
    vecina=genera_vecina_aleatorio(solucion_referencia)
    
    #Calcula distancia
    distancia_vecina=distancia_total(vecina, problem)

    #Si es la mejor solución de todas se guarda(siempre)
    if distancia_vecina < mejor_distancia:
        mejor_solucion=vecina
        mejor_distancia=distancia_vecina
    
    #Si la nueva vecina es mejor se cambia y si es peor se cambia según función de probabilidad
    if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina)):
      solucion_referencia=vecina
      distancia_referencia=distancia_vecina

    #Baja temperatura
    TEMPERATURA=bajar_temperatura(TEMPERATURA)

  print("La mejor solución encontrada es " , end="")
  print(mejor_solucion)
  print("con una distancia total de " , end="")
  print(mejor_distancia)
  return mejor_solucion

In [104]:
problem=tsplib95.load(file)
sol=recocido_simulado(problem,10000000)

La mejor solución encontrada es [0, 1, 6, 5, 14, 16, 15, 37, 19, 13, 8, 9, 29, 30, 35, 36, 17, 31, 7, 26, 18, 4, 3, 27, 2, 28, 39, 23, 41, 10, 25, 11, 12, 32, 20, 34, 33, 38, 22, 24, 40, 21]
con una distancia total de 1949


SALEN MAS CORTAS(OPTIMAS) LAS TRAYECTORIAS(SOLUCIONES) DE SIMULATED ANNEALING!

5. COLONIA DE HORMIGAS

In [105]:
#DATOS YA CARGADOS EN LOS ANTERIORES EJERCICIOS
#FUNCIONES:
def Add_Nodo(problem, H ,T ) :
  #Mejora:Establecer una funcion de probabilidad para 
  # añadir un nuevo nodo dependiendo de los nodos mas cercanos y de las feromonas depositadas
  Nodos = list(problem.get_nodes())
  return random.choice(   list(set(range(1,len(Nodos))) - set(H) )  )

def Incrementa_Feromona(problem, T, H ) :
  #Incrementa segun la calidad de la solución. Añadir una cantidad inversamente proporcional a la distancia total 
  for i in range(len(H)-1):
    T[H[i]][H[i+1]] += 1000/distancia_total(H, problem)
  return T

def Evaporar_Feromonas(T ):
  #Evapora 0.3 el valor de la feromona, sin que baje de 1
  #Mejora:Podemos elegir diferentes funciones de evaporación dependiendo de la cantidad actual y de la suma total de feromonas depositadas,...
  T = [[ max(T[i][j] - 0.3 , 1) for i in range(len(Nodos)) ] for j in range(len(Nodos))]
  return T

In [106]:
def hormigas(problem, N) :

  #Nodos
  Nodos = list(problem.get_nodes())
  #Aristas
  Aristas = list(problem.get_edges()) 
  
  #Inicializa las aristas con una cantidad inicial de feromonas:1
  #Mejora: inicializar con valores diferentes dependiendo diferentes criterios
  T = [[ 1 for _ in range(len(Nodos)) ] for _ in range(len(Nodos))]
  
  #Se generan los agentes(hormigas) que serán estructuras de caminos desde 0
  Hormiga = [[0] for _ in range(N)]
  
  #Recorre cada agente construyendo la solución
  for h in range(N) :
    #Para cada agente se construye un camino
    for i in range(len(Nodos)-1) :
      
      #Elige el siguiente nodo
      Nuevo_Nodo = Add_Nodo(problem, Hormiga[h] ,T )
      Hormiga[h].append(Nuevo_Nodo)     
    
    #Incrementa feromonas en esa arista 
    T = Incrementa_Feromona(problem, T, Hormiga[h] )
    #print("Feromonas(1)", T)
      
    #Evapora Feromonas  
    T = Evaporar_Feromonas(T)
    #print("Feromonas(2)", T)

    #Seleccionamos el mejor agente
  mejor_solucion = []
  mejor_distancia = 10e100
  for h in range(N) :
    distancia_actual = distancia_total(Hormiga[h], problem)
    if distancia_actual < mejor_distancia:
      mejor_solucion = Hormiga[h]
      mejor_distancia =distancia_actual

  print(mejor_solucion)
  print(mejor_distancia)
  
hormigas(problem, 1000)

[0, 28, 17, 27, 31, 20, 7, 6, 37, 26, 16, 23, 8, 39, 9, 18, 19, 32, 24, 38, 30, 36, 33, 35, 34, 41, 29, 21, 22, 2, 3, 12, 11, 13, 25, 5, 10, 15, 14, 40, 4, 1]
3842


In [107]:
hormigas(problem, 10000)

[0, 17, 30, 33, 20, 34, 38, 16, 15, 3, 5, 41, 10, 11, 1, 31, 35, 26, 4, 6, 36, 24, 40, 25, 23, 21, 27, 28, 29, 9, 12, 2, 18, 7, 19, 39, 32, 14, 37, 13, 22, 8]
3681


AUMENTANDO EL NÚMERO DE HORMIGAS MEJORA LA SOLUCIÓN, PERO NO SUPERA A SIMULATED ANNEALING. PUEDE QUE CON MÁS HORMIGAS SÍ, PERO EL CÁLCULO ES MUY COSTOSO...